In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob

from astropy.table import Table, join, Column
% matplotlib inline

import matplotlib as mpl
mpl.style.use('seaborn-colorblind')

#To make sure we have always the same matplotlib settings
#(the ones in comments are the ipython notebook settings)

mpl.rcParams['figure.figsize']=(12.0,9.0)    #(6.0,4.0)
mpl.rcParams['font.size']=20               #10 
mpl.rcParams['savefig.dpi']= 200             #72 
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
from matplotlib import rc

colours = mpl.rcParams['axes.prop_cycle'].by_key()['color']


In [ ]:
fname = '../data/haloC4.csv'
data = Table.read(fname,format='ascii')
shortcad = open('../data/shortcad.txt','r').read().split('\n')

In [ ]:
data

In [ ]:
campaigns = range(4,19)
objective = 'tv'
lag = 10
template = "wget -q -nH --cut-dirs=6 -r -l0 -c -N -np -R 'index*' -erobots=off"

f = open('../data/download_halo.txt','w')
h = open('../scripts/sbatch_k2sc_all.txt','w')
h.write('python correct_halo.py -c 6 -name Spica --objective %s &> Spica_output.txt\n' % objective) # do Spica manually
k = open('../reduced/sbatch_halo_all.txt','w')
short = open('../reduced/short.txt','w')
for campaign in campaigns:
    if campaign == 5:
        continue
    print('Campaign',campaign,'\n')
    g = open('../reduced/halo_commands_c%d.txt' % campaign,'w')
    fname = '../data/haloC%d.csv' % campaign
    print(fname)
    data = Table.read(fname,format='ascii')
    
    for star in data:
        epic = star['EPIC ID']
        if 'alo' in star['MASK']:
            print(star['Name'],star['EPIC ID'],star['MASK'])
            if campaign == 11:
                campaign = 111 # no 112
            fname = 'ktwo%d-c%02d_lpd-targ.fits.gz' % (epic,campaign)
            shortfname = fname.replace('lpd','spd')
            if campaign in [9, 10]:
                for j in [1,2]:
                    fname = 'ktwo%d-c%d%d_lpd-targ.fits.gz' % (epic,campaign,j)
                    url = 'https://archive.stsci.edu/pub/k2/target_pixel_files/c%d%d/%s00000/%s000/%s' % (campaign,j,str(epic)[:4],str(epic)[4:6],fname)
                    command = template+' '+url
                    f.write(command+'\n')
                    halocommand = 'halo %s --data-dir ../data/ --save-dir ../reduced/c%d%d/ --name %s -c %d%d --thresh 0.5 --objective %s --lag %d --do-plot\n' % (fname,campaign,j,star['Name'].replace(' ','_'),campaign,j,objective,lag)
                    if campaign ==9:
                        halocommand = 'halo %s --data-dir ../data/ --save-dir ../reduced/c%d%d/ --name %s -c %d%d --thresh 0.5 --objective %s --lag %d --deathstar --do-plot\n' % (fname,campaign,j,star['Name'].replace(' ','_'),campaign,j,objective,lag)
                    g.write(halocommand)
                    k.write(halocommand)
                    if shortfname in shortcad:
                        shortcommand = 'halo %s --data-dir ../data/ --save-dir ../reduced/short/ --name %s -c %d%d --thresh 0.5 --objective %s --lag 300 --deathstar --do-plot\n' % (shortfname,star['Name'].replace(' ','_'),campaign,j,objective)
                        short.write(shortcommand)

                    if (campaign == 10) and (j == 2):
                        k2sc_command = 'python correct_halo.py -c %d%d -name %s --objective %s &> %s_output%d.txt' % (campaign,j,star['Name'].replace(' ','_'),objective,star['Name'].replace(' ','_'),j)
                        h.write(k2sc_command+'\n')
            else:
                url = 'https://archive.stsci.edu/pub/k2/target_pixel_files/c%d/%s00000/%s000/%s' % (campaign,str(epic)[:4],str(epic)[4:6],fname)
                command = template+' '+url
                f.write(command+'\n')
                k2sc_command = 'python correct_halo.py -c %d -name %s --objective %s &> %s_output.txt' % (campaign,star['Name'].replace(' ','_'),objective,star['Name'].replace(' ','_'))
                h.write(k2sc_command+'\n')

#             print(command)
                halocommand = 'halo %s --data-dir ../data/ --save-dir ../reduced/c%d/ --name %s -c %d --thresh 0.5 --objective %s --lag %d --do-plot\n' % (fname,campaign,star['Name'].replace(' ','_'),campaign,objective,lag)
                g.write(halocommand)
                k.write(halocommand)
                if shortfname in shortcad:
                    shortcommand = 'halo %s --data-dir ../data/ --save-dir ../reduced/short/ --name %s -c %d --thresh 0.5 --objective %s --lag 300 --do-plot\n' % (shortfname,star['Name'].replace(' ','_'),campaign,objective)
                    short.write(shortcommand)

#             print(halocommand)
    g.close()
    print('\n')
f.close()
h.close()
k.close()
short.close()

In [ ]:
fname.replace('lpd','lpd')

### Now let us make some tables for the paper


In [ ]:
data[['Halo' in mask for mask in data['MASK']]]

In [ ]:
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy import units as u



In [ ]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('sptype')
sim = Simbad.query_objects(data['Name'])
data['SpType'] = sim['SP_TYPE']


In [ ]:
from astropy.io import ascii
names = ['Name','EPIC','RA (J2000)','Dec (J2000)','Spectral','V']
units = ['', '', '(deg)','(deg)','Type','mag']

latexdict = ascii.latex.latexdicts['AA'].copy()
latexdict['tabletype']= 'table*'
latexdict['units'] = dict(zip(names,units))


In [ ]:
dwarf_variability = Table.read('../data/dwarf_variability.csv',format='ascii')

In [ ]:
rgs = Table.read('../data/TargetsSeisParamsShort.csv',format='ascii')

In [ ]:
rgs

In [ ]:
to_write = data[['Halo' in mask for mask in data['MASK']]]['Name','EPIC ID','RA (J2000) [deg]','Dec (J2000) [deg]','SpType','Vmag']
to_write.rename_column('EPIC ID','EPIC')
to_write.rename_column('RA (J2000) [deg]','RA (J2000)')
to_write.rename_column('Dec (J2000) [deg]','Dec (J2000)')
to_write.rename_column('SpType','Spectral')
to_write.rename_column('Vmag','V')
to_write['Campaign'] = campaign*np.ones(len(to_write),dtype='int')


caption = r"All stars observed with halo photometry in Campaign %d." % campaign 
to_write.write('../paper/table_c%d.tex' % campaign,format='latex',latexdict=latexdict,caption=caption+'\label{table_c%d}' % campaign,overwrite=True)
print('Written %s' % '../paper/table_c%d.tex' % campaign)


In [ ]:
from astropy.table import vstack, hstack
simgreek = ['alf','bet','gam','del','eps','zet','eta','tet','iot','kap','lam','mu','nu','ksi','omi','pi','rho','sig','tau','ups','phi','khi','psi','ome']
latexgreek = ['\\alpha', '\\beta', '\\gamma','\\delta','\\epsilon','\\zeta','\\eta','\\theta','\\iota','\\kappa','\\lambda','\\mu','\\nu','\\xi','o','\\pi','\\rho','\\sigma','\\tau','\\upsilon','\\phi','\\chi','\\psi','\\omega']
def translate_greek(word):
    for j, letter in enumerate(simgreek):
        if letter in word:
            word = word.replace(letter,'$%s$' % latexgreek[j])
            return(word)
    return(word)

In [ ]:
to_write['Name']

In [ ]:
star['Name']

In [ ]:
campaigns = range(4,19)

campaign_tables = []

for campaign in campaigns:
    if campaign == 5:
        continue
    print('Campaign',campaign,'\n')
    
    fname = '../data/haloC%d.csv' % campaign
    data = Table.read(fname,format='ascii')
    data=data[~data['Name'].mask]
    
    sc = data[['SC' in mask for mask in data['MASK']]]
    data = data[['SC' not in mask for mask in data['MASK']]]
    
    sim = Simbad.query_objects(data['Name'])
    if campaign == 4:
        sim = sim[sim['COO_ERR_MAJA']<10]
    
    data['SpType'] = Column([sptype.replace('_',' ') for sptype in sim['SP_TYPE'].astype('str')],name='SpType')
    data['Campaign'] = campaign*np.ones(len(data),dtype='int')
    
    if campaign != 6:
        to_write = data[['Halo' in mask for mask in data['MASK']]]['Name','EPIC ID','SpType','Vmag','Campaign','Comments']
    else:
        to_write = data['Name','EPIC ID','SpType','Vmag','Campaign','Comments']
    to_write['Comments']= to_write['Comments'].astype('str')
    for short in sc['Name']:
        to_write['Comments'][to_write['Name']==short] = 'SC' 
    to_write.rename_column('EPIC ID','EPIC')
    to_write.rename_column('Comments','Notes')
#     to_write.rename_column('RA (J2000) [deg]','RA (J2000)')
#     to_write.rename_column('Dec (J2000) [deg]','Dec (J2000)')
    to_write.rename_column('SpType','Spectral')
    to_write.rename_column('Vmag','V')
    variability = []
    for star in to_write:
        name = star['Name'].replace(' ','_')
        epic = star['EPIC']
        if name in dwarf_variability['Name']:
            index = np.where(dwarf_variability['Name']==name)[0]
            var = dwarf_variability['Variability'][index].data.data[0]
            if var == 'a2CVn':
                var = '$\\alpha^2$\\,CVn'
            print('%s is a %s variable' % (name,str(var)))
            variability.append(var)
        elif epic in rgs['epicid']:
            var = 'RG'
            print('%s is a %s variable' % (name,var))
            variability.append(var)
        else:
            print('%s is not variable' % (name))
            variability.append('--')
    to_write.add_column(Column(variability,name='Class'))
    campaign_tables.append(to_write)
    
all_write = vstack(campaign_tables)
all_write['Campaign'] = all_write['Campaign'].astype('str')

repeated = set([i for i in all_write['Name'] if sum([1 for a in all_write['Name'] if a == i]) > 1])
for repeat in repeated:
    print(repeat,'repeated')
    m = (all_write['Name']==repeat)
    all_write[np.where(m)[0][0]]['Campaign'] += ', %s' % all_write[m]['Campaign'][1]
    all_write.remove_row(np.where(m)[0][1])
    
for j, name in enumerate(all_write['Name']):
    if name[0].isupper():
        continue
    else:
        all_write['Name'][j] = translate_greek(name)
    
units = ['','','Type','(mag)','','']
latexdict_apj = {'units':dict(zip(all_write.keys(),units))}
caption = r"All stars observed with halo photometry in K2."
all_write.write('../paper/table_all.tex',format='aastex',caption=caption+'\label{table_all}',
                latexdict=latexdict_apj,overwrite=True)
print('Written %s' % '../paper/table_all.tex')
tablenotes = ['\\tablenotetext{a}{\citet{White2017}}',
              '\\tablenotetext{b}{\citet{Buysschaert2018}}',
              '\\tablenotetext{c}{\citet{Farr2018}}',
              '\\tablenotetext{d}{\citet{Arentoft2019}}',
              '\\tablenotetext{e}{\citet{Aerts2018}}',
             '\\tablenotetext{1}{abCN0.5}']
with open('../paper/table_all.tex', 'r') as original: oldtext = original.read().replace('\end{deluxetable}',
                                                                                        '\n'.join(tablenotes)+
                                                                                        '\n\end{deluxetable}')
with open('../paper/table_all.tex', 'w') as modified: modified.write("\\startlongtable\n" + oldtext)


In [ ]:
tablenotes = ['\tablenotetext{a}{\citet{White2017}}']*5
'\n'.join(tablenotes)

In [ ]:
for l, ccat in enumerate([all_write[:29],all_write[29:54],all_write[54:81],all_write[81:114],all_write[114:134],all_write[134:]]):
    latexdict = ascii.latex.latexdicts['AA'].copy()
    latexdict['tabletype']= 'table*'
    latexdict['units'] = dict(zip(names,units))
    
    campaigns_list = ['Campaigns 7-8','Campaign 9','Campaigns 10-12','Campaign 13','Campaigns 14-15','Campaigns 16-18']

    caption = "Stars in %s observed with halo photometry in K2." % campaigns_list[l]
    ccat.write('../paper/all_stars_%d.tex' % (l+1),format='latex',latexdict=latexdict,caption=caption+'\label{table_%s}' % l,overwrite=True)
    print('Written %s' % '../paper/all_stars_%d.tex' % (l+1))
        


### Which are giants and which are main sequence?


In [ ]:
Simbad.reset_votable_fields()
# Simbad.list_votable_fields()
Simbad.add_votable_fields('parallax')
sim = Simbad.query_objects(data['Name'])
# data['SpType'] = sim['SP_TYPE']


In [ ]:
def abs_mag(app_mag,dist):
    return app_mag - 5.*(np.log10(dist) - 1.)

In [ ]:
abs_V = abs_mag(data['Vmag'],1./(sim['PLX_VALUE']))

In [ ]:
campaigns = range(6,19)

campaign_tables = []

Simbad.add_votable_fields('parallax')
Simbad.add_votable_fields('parallax')
Simbad.add_votable_fields('flux(B)')
Simbad.add_votable_fields('sptype')


for campaign in campaigns:
    print('Campaign',campaign,'\n')
    
    fname = '../data/haloC%d.csv' % campaign
    data = Table.read(fname,format='ascii')
    data=data[~data['Name'].mask]

    data = data[['SC' not in mask for mask in data['MASK']]]
    sim = Simbad.query_objects(data['Name'])
    data['SpType'] = Column([sptype.replace('_',' ') for sptype in sim['SP_TYPE'].astype('str')],name='SpType')

    data['parallax'] = Column([parallax for parallax in sim['PLX_VALUE']],name='parallax')
    data['Bmag'] = Column([b for b in sim['FLUX_B']],name='Bmag')

    data['Campaign'] = campaign*np.ones(len(data),dtype='int')
            
    to_write = data[['Halo' in mask for mask in data['MASK']]]['Name','EPIC ID','Vmag','Bmag','Campaign','parallax','SpType']
    to_write.rename_column('EPIC ID','EPIC')
    campaign_tables.append(to_write)
    
all_plot = vstack(campaign_tables)
all_plot['Campaign'] = all_plot['Campaign'].astype('str')
all_plot['abs_Vmag'] = abs_mag(all_plot['Vmag'],1./(all_plot['parallax']/1000.))
all_plot['b_v'] = all_plot['Bmag']-all_plot['Vmag']

In [ ]:
giants = np.array(['III' in sptype for sptype in all_plot['SpType']])

In [ ]:
plt.plot(all_plot['b_v'],all_plot['abs_Vmag'],'.')
plt.plot(all_plot['b_v'][giants],all_plot['abs_Vmag'][giants],'.')

plt.ylim(4,-8)
plt.axvline(0.75)
plt.axvline(0.5,color=colours[2])

In [ ]:
funny = (all_plot['b_v']<0.75)*(all_plot['b_v']>0.5)
all_plot[funny]

In [ ]:
# psi Sgr is a giant
# 42 Cet is a subgiant
# 7 Sgr is a giant

In [ ]:
Table(all_plot['Name','EPIC'][all_plot['b_v']>0.5]).write('halo_giants.csv')
Table(all_plot['Name','EPIC'][all_plot['b_v']<0.5]).write('halo_dwarfs.csv')


In [ ]:
# tab.write('halo_giants.csv')

In [ ]:
write_rgs = rgs['epicid',]
write_rgs.rename_column('epicid','EPIC')

new_numax = Column(['%.1f $\\pm$ %.2f' % (numax, rgs['numaxerr'][j]) for j,numax in enumerate(rgs['numax'])],name='\\numax')
new_Dnu = Column(['%.1f $\\pm$ %.2f' % (Dnu, rgs['dnuerr'][j]) for j,Dnu in enumerate(rgs['dnu'])],name="\\Dnu")
write_rgs.add_column(new_numax)
write_rgs.add_column(new_Dnu)
write_rgs = join(all_write['Name','EPIC'],write_rgs,keys='EPIC')
units = ['','','(\\muHz)','(\\muHz)']
latexdict_rgs = {'units':dict(zip(write_rgs.keys(),units))}

caption='Global asteroseismic parameters for the 31 red giants for which solar-like oscillations were detected.'
write_rgs.write('../paper/rgs.tex',format='aastex',caption=caption+'\label{rgs}',
                latexdict=latexdict_rgs,overwrite=True)

# with open('../paper/rgs.tex', 'r') as original: oldtext = original.read()
# with open('../paper/rgs.tex', 'w') as modified: modified.write("\\startlongtable\n" + oldtext)
join(all_write['Name','EPIC'],write_rgs,keys='EPIC')